# RA2CE feature: Isolated locations
**NOTE: TEXT WILL BE UPDATED**

This notebook contains an example of the **isolated locations analysis** of the RA2CE model.

First of all, we will import the packages we need to execute the notebook:

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
from ra2ce.ra2ce_handler import Ra2ceHandler #import the ra2cehandler to run ra2ce analyses

First, we will set the path to the ra2ce folder and we will initialize the names for the network and analysis ini files. Afterwards, let's find the examples data folder.

In [ ]:
root_dir = Path().parent / "data" / "isolated_locations"  # set the root directory for the data

network_ini = root_dir / "network.ini"
assert network_ini.is_file()

analyses_ini = root_dir / "analyses.ini"
assert analyses_ini.is_file()

# Isolated locations analysis
EXPLANATION

**Network.ini content**
<br>
<br>
> [project]<br>
name = isolated_locations<br>
<br>
[network]<br>
directed = False<br>
source = OSM download<br>
primary_file = None<br>
diversion_file = None<br>
file_id = None<br>
polygon = map.geojson<br>
network_type = drive<br>
road_types = motorway,motorway_link,trunk,trunk_link,primary,primary_link,secondary,secondary_link,tertiary,tertiary_link<br>
save_gpkg = False<br>
<br>
[origins_destinations]<br>
origins = None<br>
destinations = None<br>
origins_names = None<br>
destinations_names = None<br>
id_name_origin_destination = None<br>
origin_count = None<br>                         
origin_out_fraction = 1<br>
category = category<br>
<br>
[hazard]<br>
**hazard_map = max_flood_depth.tif**<br>
hazard_id = None<br>
hazard_field_name = waterdepth<br>
**aggregate_wl = max**<br>
**hazard_crs = EPSG:32736**<br>
<br>
*[cleanup]  # Can only be used if a network is created from a shapefile<br>
snapping_threshold = None<br>
segmentation_length = None<br>
merge_lines = False<br>
merge_on_id = False<br>
cut_at_intersections = False<br>*

Configure the network

In [ ]:
handler = Ra2ceHandler(network=network_ini, analysis=analyses_ini)
handler.configure()

**Analyses.ini content**
<br>
<br>
> [project]<br>
name = isolated_locations<br>
<br>
[analysis1]<br>
name = multilink isolated locations<br>
analysis = multi_link_isolated_locations<br>
aggregate_wl = max<br>
threshold = 1<br>
weighing = length<br>
category_field_name = category<br>
save_gpkg = True<br>
save_csv = True<br>

In [ ]:
handler.run_analysis()